## Understanding of Image Segmentation

Suppose you want to know the shape of the object in an image, which pixel belongs to which object, etc, you need to assign a class to each pixel of the image - this task is known as segmentation.

A segmented model returns much more detailed information about the image. Some applications of image segmentation are in medical imaging, self-driving cars and satellite imaging, just to name a few.

This project uses the OXford-IIIT Pet Dataset(Parkhi et al, 2012). The dataset consists of images of 37 pet breeds, with 200 image per breed(~100 each in the training and test splits). Each image includes the corresponding labels, and pixel-wise masks. The masks are class-labels for each pixel. Each pixel is given one of three categories:
    
    1. Class 1: Pixel belonging to the pet
    
    2. Class 2: Pixel bordering the pet.
    
    3. Class 3: None of the above/a surrounding pixel